In [352]:
import requests
import pandas as pd

api_key = 'YF0US57AOZXKGIGD'



In [353]:
class script_data:
    def __init__(self, symbol):
        self.symbol = symbol
        # self.interval = interval
        self.url = f'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={symbol}&interval=5min&apikey={api_key}'

    def fetch_intraday_data(self):
        response = requests.get(self.url)
        if response.status_code == 200:
            d = response.json()
            del d['Meta Data']
            return d
        else:
            raise Exception(
                f"Failed to fetch data. Response code: {response.status_code}")
    
    def convert_intraday_data(self, script):
        data = {}
        data["timestamp"] = []
        data["open"] = []
        data["high"] = []
        data["low"] = []
        data["close"] = []
        data["volume"] = []
        index = 1
        for key, value in script.items():
            if key.startswith('Time Series (5min)'):
                for key, prices in value.items():
                    data["timestamp"].append(str(key))
                    data["open"].append(float(prices["1. open"]))
                    data["high"].append(float(prices["2. high"]))
                    data["low"].append(float(prices["3. low"]))
                    data["close"].append(float(prices["4. close"]))
                    data["volume"].append(int(prices["5. volume"]))
        df = pd.DataFrame(data)
        return df
    
    def __getitem__(self, item):
        return self.df[item]
    
    def __setitem__(self, key, value):
        self.df[key] = value
    
    def __contains__(self, item):
        return item in self.df.columns

    def indicator1(df, timeperiod):
        result = df.copy()
        result["indicator"] = result["close"].rolling(window=timeperiod).mean()
        return result[["timestamp", "indicator"]]

        
class Strategy(script_data):
    def __init__(self, script):
        self.script = script
        self.df = None
        self.indicator_data = None
        # self.script_data = script_data("GOOGL")
        self.url = f'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={script}&interval=5min&apikey={api_key}'

    def indicator2(df, timeperiod):
        result = df.copy()
        result["indicator_data"] = result["close__data"].rolling(window=timeperiod).mean()
        return result[["timestamp", "indicator_data"]]

    
    # def get_script_data():
    #     data = sd.fetch_intraday_data()
    #     self.df = script_data.convert_intraday_data(data)
    #     result = indicator1(self.df, timeperiod=5)
    #     return result
        
    # def get_signal(script):
    #     data = {}
    #     data["timestamp"] = []
    #     data["signal"] = []
    #     for key, value in script.items():
    #         if key.startswith('Time Series (5min)'):
    #             for key, prices in value.items():
    #                 data["timestamp"].append(str(key))

    #                 data["signal"].append(int(prices["5. volume"]))
    #     df = pd.DataFrame(data)
    #     return df


        


In [354]:
sd = script_data(symbol="GOOGL")
data = sd.fetch_intraday_data()
# print(data)




In [355]:
df = sd.convert_intraday_data(data)
print(df)


              timestamp     open   high    low    close  volume
0   2023-02-13 20:00:00  94.3500  94.40  94.28  94.4000   13261
1   2023-02-13 19:55:00  94.3500  94.40  94.35  94.3500    6407
2   2023-02-13 19:50:00  94.2800  94.39  94.28  94.3500    2081
3   2023-02-13 19:45:00  94.4200  94.44  94.29  94.2900    2133
4   2023-02-13 19:40:00  94.4700  94.47  94.35  94.3500     917
..                  ...      ...    ...    ...      ...     ...
95  2023-02-13 12:05:00  94.2900  94.31  94.22  94.2900  398928
96  2023-02-13 12:00:00  94.2701  94.31  94.24  94.3000  467125
97  2023-02-13 11:55:00  94.2500  94.30  94.12  94.2725  553289
98  2023-02-13 11:50:00  94.3064  94.36  94.23  94.2500  503563
99  2023-02-13 11:45:00  94.0300  94.41  94.02  94.3021  611545

[100 rows x 6 columns]


In [356]:
sd.df = df
print("open" in sd)


True


In [357]:
result = indicator1(df, timeperiod=5)
print(result)

              timestamp  indicator
0   2023-02-13 20:00:00        NaN
1   2023-02-13 19:55:00        NaN
2   2023-02-13 19:50:00        NaN
3   2023-02-13 19:45:00        NaN
4   2023-02-13 19:40:00   94.34800
..                  ...        ...
95  2023-02-13 12:05:00   94.37956
96  2023-02-13 12:00:00   94.34656
97  2023-02-13 11:55:00   94.31106
98  2023-02-13 11:50:00   94.28206
99  2023-02-13 11:45:00   94.28292

[100 rows x 2 columns]


In [358]:
st = Strategy("GOOGL")
data1 = st.fetch_intraday_data()
df1 = st.convert_intraday_data(data1)
df1 = df.rename(columns={'close': 'close_data'})
print(df1)


              timestamp     open   high    low  close_data  volume
0   2023-02-13 20:00:00  94.3500  94.40  94.28     94.4000   13261
1   2023-02-13 19:55:00  94.3500  94.40  94.35     94.3500    6407
2   2023-02-13 19:50:00  94.2800  94.39  94.28     94.3500    2081
3   2023-02-13 19:45:00  94.4200  94.44  94.29     94.2900    2133
4   2023-02-13 19:40:00  94.4700  94.47  94.35     94.3500     917
..                  ...      ...    ...    ...         ...     ...
95  2023-02-13 12:05:00  94.2900  94.31  94.22     94.2900  398928
96  2023-02-13 12:00:00  94.2701  94.31  94.24     94.3000  467125
97  2023-02-13 11:55:00  94.2500  94.30  94.12     94.2725  553289
98  2023-02-13 11:50:00  94.3064  94.36  94.23     94.2500  503563
99  2023-02-13 11:45:00  94.0300  94.41  94.02     94.3021  611545

[100 rows x 6 columns]


In [360]:
result1 = st.indicator2(df1, timeperiod=5)
print(result1)


TypeError: Strategy.indicator2() got multiple values for argument 'timeperiod'